# Download and downscale ERA5 daily products using [TopoPyScale](https://topopyscale.readthedocs.io/en/latest/)

In [ ]:
import pandas as pd
from TopoPyScale import topoclass as tc
from matplotlib import pyplot as plt
from TopoPyScale import topo_plot as plot
import os
import xarray as xr
import rioxarray as rxr
import numpy as np

In [ ]:
# ========= STEP 1 ==========
# Load Configuration
config_file = '/Volumes/LaCie/raineyaberle/Research/PhD/SnowMaL/config.yml'
mp = tc.Topoclass(config_file)
# Compute parameters of the DEM (slope, aspect, sky view factor)
mp.compute_dem_param()

In [ ]:
# ======== STEP 2 ===========
# Check if downscaled climate data already exist in file
out_fn = os.path.join(mp.config['outputs']['directory'], 'output.nc')
if not os.path.exists(out_fn):
    # Run the downscaling pipeline
    # Calculate topographic variables, generate clusters
    # Note: I had to delete the ds_param.nc file before the next step would run
    ds_param_fn = os.path.join(mp.config['outputs']['directory'], mp.config['outputs']['file']['ds_param'])
    os.remove(ds_param_fn)
    mp.extract_topo_param()
    # compute solar geometry and horizon angles
    mp.compute_solar_geometry()
    mp.compute_horizon()
    # Perform the downscaling
    mp.downscale_climate()
    # Export output to netCDF
    mp.to_netcdf()
    # Plot temp in K for one time step
    plot.map_variable(mp.downscaled_pts, mp.toposub.ds_param, time_step=100, var='t')
    plt.show()

# Load output downscaled climate data
out = xr.open_dataset(out_fn)
print('Downscaled climate data loaded from file.')

In [ ]:
# ========= STEP 3 ==========
# Convert downscaled climate data into daily mean temperature and precip time series
out_adj_fn = out_fn.replace('.nc', '_daily.nc')
if os.path.exists(out_adj_fn):
    out_adj_spatial = xr.open_dataset(out_adj_fn)
    print('Daily climate and terrain variables loaded from file.')
else:

    # Adjust downscaled climate variables
    # Air temperature: convert to degrees C, take the mean daily value
    out_temp_C = (out['t'] - 273.15).resample(time='1D').mean().assign_attrs(units='C', long_name='Temperature', standard_name='temperature')
    # Precipitation: take the daily sum of values (mm/hr --> mm/d)
    out_precip = out['tp'].resample(time='1D').sum().assign_attrs(units='mm d**-1', long_name='Precipitation', standard_name='precipitation')
    # Create new dataset from adjusted climate variables
    out_adj = xr.Dataset(
        data_vars=dict(
            t=out_temp_C,
            tp=out_precip,
        ),
        coords=out_temp_C.coords,
    )
    # Map back onto spatial coordinates
    out_adj_spatial = out_adj.sel(point_name=mp.toposub.ds_param.point_name)
    # Add terrain variables
    terrain_params = ['elevation', 'slope', 'aspect', 'svf']
    for param in terrain_params:
        out_adj_spatial[param] = mp.toposub.ds_param[param]
    # Set points with no elevation data to NaN
    out_adj_spatial = xr.where(out_adj_spatial['elevation'] == out_adj_spatial['elevation'].data[0,0], np.nan, out_adj_spatial)
    # Set spatial information
    dem_fn = os.path.join(mp.config['dem']['path'], mp.config['dem']['file'])
    crs = f'EPSG:{rxr.open_rasterio(dem_fn).rio.crs.to_epsg()}'
    out_adj_spatial = out_adj_spatial.rio.write_crs(crs)
    # Save to file
    out_adj_spatial.to_netcdf(out_adj_fn)
    print('Daily climate and terrain variables saved to file:', out_adj_fn)

# Plot mean temperature to check results
out_adj_spatial['t'].mean(dim='time').plot.imshow()
plt.show()